In [8]:
import os

import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.pipeline import Pipeline

In [9]:
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

pipeline_session = PipelineSession()

In [10]:
transformer = Transformer(
    model_name="pipelines-rbidfcjqafy5-MyModelCreationStep--lHX2aEB7wQ",
    instance_type="ml.m5.large",
    instance_count=1,
    output_path=f"s3://sagemaker-bucket-ds/PIPELINE/04-simple-batch-transform/"
)

In [11]:
step_transform = TransformStep(
    name="MyTransformStep",
    transformer=transformer,
    inputs=TransformInput(
        data="s3://sagemaker-bucket-ds/PIPELINE/01_simple_processing_pipeline/iris_inference.csv",
        content_type='text/csv', # It is neccessary because csv is not default format
        split_type='Line' # Each line equals one observation)
))

In [12]:
pipeline_name = f"04-simple-batch-transform"
pipeline = Pipeline(
    name=pipeline_name,
    steps=[step_transform],
)

In [13]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:211125740051:pipeline/04-simple-batch-transform',
 'ResponseMetadata': {'RequestId': 'f617a0d4-4a0c-452a-95c0-d91493640e04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f617a0d4-4a0c-452a-95c0-d91493640e04',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Thu, 11 Jul 2024 12:59:08 GMT'},
  'RetryAttempts': 0}}

In [14]:
execution = pipeline.start()